## Get some data

In [1]:
import temp
import model as trf

In [2]:
vocab = 40000

In [62]:
b = temp.get_some_batch()

In [63]:
b

tensor([    2,    14,    28,    15,    36,    12,    95,   455,     9,   143,
          119,     8,  1293,    12, 18593,   134,   101,    10,   158,    80,
          103,  3658,     9,  2432,    47, 18703,     7, 15800,     3])

In [98]:
b = b.unsqueeze(-2) #batch로 만들기

In [83]:
x = temp.emb_pe(b, vocab)

In [84]:
x.shape

torch.Size([1, 29, 128])

In [7]:
softmax_ = trf.Generator(128, vocab)

In [8]:
input_ = softmax_(x)

In [9]:
input_.shape

torch.Size([1, 23, 40000])

## Train Part

In [2]:
import torch
import torch.nn as nn
import math
import numpy as np
import copy
import torch.nn.functional as F
from torch.autograd import Variable
import time

from embedding import *

In [12]:
max_src_in_batch = 40
max_tgt_in_batch = 40

In [218]:
class Batch:
    def __init__(self, src, trg=None, pad=1):
        self.src = src
        self.src_mask = self.make_std_mask(self.src, pad)
        if trg is not None:
            self.trg = trg[:, :-1]  # input으로 사용할때
            self.trg_y = trg[:, 1:] # 최종 정답으로 사용할때
            self.trg_mask = self.make_sub_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
    
    @staticmethod
    def make_sub_mask(tgt, pad=1):
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(
            trf.subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask
    
    @staticmethod
    def make_std_mask(src, pad=1):
        src_mask = (src != pad).unsqueeze(-2)
        src_mask = src_mask & Variable(
            torch.ones(src.size(1), src.size(1)).type_as(src_mask.data)) \
        & src_mask.transpose(-2,-1)
        return src_mask

In [208]:
V = 11
dummy = data_gen(11, 4, 1)

In [212]:
for i, batch in enumerate(dummy):
    batch

In [213]:
src = batch.src
trg = batch.trg
trg_y = batch.trg_y
src_mask = batch.src_mask
trg_mask = batch.trg_mask

In [214]:
src_mask

tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], dtype=torch.uint8)

In [195]:
trg_mask

tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 0],
        

In [181]:
pad = 1
src_mask = (src != pad).unsqueeze(-2).type_as(src.data) 

In [116]:
src_mask.shape

torch.Size([4, 1, 10])

In [104]:
score.masked_fill(b_mask == 0, -1e9)

tensor([[[ 1.7524e+04, -1.1778e+03,  3.1454e+02, -1.7570e+03, -1.8875e+01,
           4.2389e+02, -4.5163e+02, -2.3499e+02, -9.8924e+02, -9.5176e+02,
          -1.3709e+03,  2.9138e+02,  2.4241e+02,  4.2537e+01,  1.5596e+03,
           2.2042e+03,  8.2435e+02, -6.2846e+02,  4.9981e+01,  2.8363e+02,
           3.0681e+02, -2.0295e+02, -2.6295e+03, -5.5517e+02, -2.0805e+03,
           7.7382e+02, -2.5679e+03,  1.3081e+03,  1.0738e+03],
         [-1.1778e+03,  1.4555e+04, -1.5120e+03, -8.1292e+02, -2.9660e+03,
           8.9928e+02, -3.4906e+03,  4.4633e+02, -3.0438e+03, -8.6248e+02,
          -3.3758e+02,  6.9399e+01,  1.2673e+03,  3.9680e+02,  1.2422e+03,
          -2.0354e+03, -2.1544e+03,  7.8515e+01, -2.5882e+03, -2.0036e+03,
          -1.0349e+03,  1.5113e+03, -3.1160e+03,  3.4759e+02,  1.4447e+02,
          -6.9577e+02,  3.5558e+02,  1.7925e+03, -1.6343e+02],
         [ 3.1454e+02, -1.5120e+03,  1.8824e+04,  7.5739e+02,  2.3973e+03,
          -7.8023e+02,  3.2932e+03,  1.2247e+03, 

In [130]:
mat = torch.ones(src.size(1), src.size(1)).type_as(src.data)

In [131]:
src_mask

tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 0, 1, 1, 1, 1, 1]]])

In [136]:
mat_ = src_mask & mat & src_mask.transpose(-2, -1)
mat_

tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1,

In [135]:
mat_ & src_mask.transpose(-2, -1)

tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1,

In [105]:
score = torch.matmul(x, x.transpose(-2,-1))
score.shape

torch.Size([1, 29, 29])

In [69]:
torch.ones(src.size(1), src.size(1))

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [64]:
tgt_mask = (b != 1).unsqueeze(-2)

In [24]:
tgt_mask = tgt_mask & Variable(model.subsequent_mask(b.size(-1)))

tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], dtype=torch.uint8)

In [66]:
tgt_mask.shape

torch.Size([1, 29])

얘 왜쓰지,,,;;;

In [7]:
global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.src) + 2)
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)

In [8]:
def run_epoch(data_iter, model, loss_compute):
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(batch.src, batch.trg, 
                            batch.src_mask, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, loss / batch.ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
    return total_loss / total_tokens

### Loss_compute
 Label Smoothing Skip
 - nn.CrossEntropyLoss(*input, *target), where *input = p_ditst, *target = target label in each rows.

In [6]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0 # Padding Value들 0으로 설정
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))

In [9]:
class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm=1):
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), 
                              y.contiguous().view(-1)) / norm
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.zero_grad()
        return loss.data * norm

CE Loss Example

In [20]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)

In [21]:
input

tensor([[-0.6863,  1.8296,  2.0636,  0.9022,  0.4992],
        [-0.9279, -0.8435, -1.1101, -0.6536, -0.1928],
        [ 0.0459,  2.0524,  0.2739,  0.0477, -1.3499]], requires_grad=True)

In [22]:
target

tensor([3, 3, 0])

In [38]:
output

tensor(2.6500, grad_fn=<NllLossBackward>)

In [12]:
size = 128
padding_idx = 1

In [15]:
criterion = nn.CrossEntropyLoss()

In [16]:
input_.shape

torch.Size([1, 50, 30000])

In [18]:
b.shape

torch.Size([50])

In [23]:
criterion(input_[0], b)

tensor(29.4953, grad_fn=<NllLossBackward>)

In [25]:
true_dist = x.data.clone()

In [26]:
true_dist.fill_(1)

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]])

In [30]:
b.unsqueeze(1)

tensor([[    2],
        [  194],
        [   28],
        [  145],
        [   15],
        [ 1074],
        [   59],
        [  130],
        [   15],
        [   26],
        [   95],
        [    8],
        [ 4443],
        [   15],
        [   26],
        [   95],
        [   41],
        [ 3684],
        [ 3486],
        [    8],
        [  314],
        [ 3486],
        [    9],
        [   66],
        [ 1253],
        [    4],
        [ 1302],
        [    5],
        [    4],
        [ 3907],
        [    5],
        [  764],
        [  916],
        [    7],
        [20609],
        [    5],
        [   68],
        [  233],
        [  345],
        [    5],
        [    4],
        [ 4189],
        [ 1413],
        [    5],
        [ 1920],
        [   10],
        [  297],
        [  102],
        [ 4389],
        [    3]])

In [31]:
true_dist.scatter_(1, b.unsqueeze(1), .1)

RuntimeError: invalid argument 3: Index tensor must either be empty or have same dimensions as output tensor at c:\n\pytorch_1559129895673\work\aten\src\th\generic/THTensorEvenMoreMath.cpp:531

### Check with synthetic data

In [3]:
from model import subsequent_mask
from train import *

In [4]:
def data_gen(V, batch, nbatches):
    "Generate random data for a src-tgt copy task."
    for i in range(nbatches):
        data = torch.from_numpy(np.random.randint(1, V, size=(batch, 10)))
        data[:, 0] = 2 # <s>
        data[:, -1] = 3 # <\s>
        src = Variable(data, requires_grad=False).type(torch.long)
        tgt = Variable(data, requires_grad=False).type(torch.long)
        yield Batch(src, tgt, 1)

In [18]:
data = torch.from_numpy(np.random.randint(1, V, size=(30, 10)))
Variable(data, requires_grad=False).type(torch.long).type()

'torch.LongTensor'

In [223]:
V = 11
dummy = data_gen(11, 4, 1)

In [18]:
lut = nn.Embedding(11, 50, 1)

In [5]:
V = 11
criterion = nn.CrossEntropyLoss()
model = trf.Transformer(V, V, N = 2, d_model = 128, d_ff = 512, h=4)
model_opt = torch.optim.Adam(model.parameters(), lr = .1, betas=(.9, .98), eps = 1e-9)

In [ ]:
model.train()
run_epoch(data_gen(V, 30, 20), model, 
          SimpleLossCompute(model.generator, criterion, model_opt))
model.eval()

print(run_epoch(data_gen(V, 30, 20), model, 
          SimpleLossCompute(model.generator, criterion, None)))

In [13]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out = model.decode(memory, src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1))
                                    .type_as(src.data)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.data[0]
        ys = torch.cat([ys, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys

model.eval()

NameError: name 'model' is not defined

In [8]:
src = Variable(torch.LongTensor([[1,2,3,4,5,6,7,8,9,10]]) )
src_mask = Variable(torch.ones(1, 1, 10) )
print(greedy_decode(model, src, src_mask, max_len=10, start_symbol=1))

tensor([[1, 5, 5, 5, 5, 5, 8, 9, 5, 5]])


In [48]:
start_symbol = 2
ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)

In [55]:
memory = model.encode(src, src_mask)

In [229]:
model.decode(memory, src_mask, Variable(ys),
            Variable(subsequent_mask(ys.size(1)).type_as(src.data)))

RuntimeError: The expanded size of the tensor (40) must match the existing size (10) at non-singleton dimension 3.  Target sizes: [1, 4, 1, 40].  Tensor sizes: [4, 1, 10, 10]

In [111]:
src.shape

torch.Size([4, 10])

In [12]:
from train import Batch

In [13]:
src_mask = Batch.make_std_mask(src, 1)

In [14]:
src_mask

tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [0, 1, 1, 1, 1, 1, 0, 0, 0, 0],
         [0, 1, 1, 1, 1, 1, 1, 0, 0, 0],
         [0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
         [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
         [0, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], dtype=torch.uint8)

### Iterator

In [ ]:
class MyIterator(data.Iterator):
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))

In [ ]:
def rebatch(pad_idx, batch):
    "Fix order in torchtext to match ours"
    src, trg = batch.src.transpose(0, 1), batch.trg.transpose(0, 1)
    return Batch(src, trg, pad_idx)

## =================================
### Simple iteration

In [10]:
from process import get_data
from model import *

In [4]:
data_path = "dataset/"
file_name = "train."

In [5]:
SRC, TGT, trn = get_data(data_path + file_name)

In [11]:
pad_idx = TGT.vocab.stoi["<pad>"]

In [ ]:
model = trf.Transformer(len(SRC.vocab), len(TGT.vocab), N = 6)
criterion = nn.CrossEntropyLoss()